In [1]:
# model test, test AE with distance loss && using different transfer function

In [9]:
import database
from dataset import Dataset
from ae_test import AutoencoderTest
import torch.optim as optim
import torch
from tqdm import tqdm_notebook
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import os
from sklearn import manifold 
# 如果想要import 子文件下的py檔，需要在子文件下建立一個__init__.py，可以爲空
# import plt.plt_hist as plt_hist
import matplotlib.pyplot as plt
import matplotlib
from ae_test import AutoencoderTest
from sklearn.externals import joblib

In [10]:
def distance_matrix_Z(data,eps=1e-8):
    x1 = data.unsqueeze(1).repeat(1, data.shape[0], 1)
    x2 = data.unsqueeze(0)
    dist = (x1 - x2).pow(2).sum(2)
    dist = (dist+eps).sqrt()
    # get the row size 
    r = data.shape[0] 
    mask = torch.triu(torch.ones([r,r]),diagonal=1)
    return dist[mask==1]

In [11]:
# 计算 batch data 两两距离,利用 mse
# 使用 transfer function
def distance_matrix_Ｘ(data,mean,eps=1e-8,k=3):
    x1 = data.unsqueeze(1).repeat(1, data.shape[0], 1)
    x2 = data.unsqueeze(0)
    dist = (x1 - x2).pow(2).sum(2)
    dist = (dist+eps).sqrt()
    r = data.shape[0]
    mask = torch.triu(torch.ones([r,r]),diagonal=1) 
    # 传统方式
    # dist[mask==1] = (dist[mask==1])/g_mean
    ## 1. test 距离小于1的,就套用(1/(X+1)**6)+1,让它的增幅变大
#     dist[mask==1] = (dist[mask==1])/1.147
#     dist[dist<=1] = (1/-(dist[dist<=1]+1)**6)+1
    # 2. test transfer function, function 为 1/(-(X+0.874)**3)+1.5,
    dist[mask==1] = 1/(-(dist[mask==1]+0.874)**k)+1.5
    dist[mask==1] = dist[mask==1]/mean
    # 3. test 线性转换
    # dist = dist*1.2
    # dist[dist>1.5] = 1.5
    # dist[mask==1] = dist[mask==1]/0.9628
    return dist[mask==1]

In [12]:
# load data
distance_matrix = torch.load('distance_map/bike_sharing_pre_compute_distance_matrix_temp.pth')
distance_matrix_mean = distance_matrix.mean()
print(distance_matrix_mean)

tensor(1.0001)


In [13]:
# model setting
device = torch.device('cuda')
BATCH_SIZE = 64
LEARNING_RATE = 1e-4

In [15]:
# 資料庫連接設定
dataset_name = 'bike_sharing'
pre_sort = 'sort/'+dataset_name+'_sort.pth'
db = database.DEFAULT_DB()
db.setUp()
# 資料集資料載入
dataset = Dataset(db,dataset_name)
dataset.set_columns(columns=['temp','hum','casual','windspeed','registered','cnt'])
dataset.load(BATCH_SIZE,1,1)
rawdata = dataset.rawdata
normalizer = dataset.normalizer
MAX_EPOCH = 100 // len(dataset.dataloader) + 1
INPUT_DIM = len(dataset.columns)
mse_each = nn.MSELoss()
normalizer_data = normalizer.transform(rawdata)
print(normalizer_data.shape)
times = int(round(dataset.rawdata.shape[0]/BATCH_SIZE))

torch.Size([17379, 6])
(17379, 6)


In [32]:
# 网络结构
def model():
    encoder = nn.Sequential(
        nn.Linear(6, 128),
        nn.Tanh(),
        nn.Linear(128, 64),
        nn.Tanh(),
        nn.Linear(64, 32),
        nn.Tanh(),
        nn.Linear(32, 8),
        nn.Tanh(),
        nn.Linear(8, 2)).to(device)
    decoder = nn.Sequential(
        nn.Linear(2, 8),
        nn.Tanh(),
        nn.Linear(8, 32),
        nn.Tanh(),
        nn.Linear(32, 64),
        nn.Tanh(),
        nn.Linear(64, 128),
        nn.Tanh(),
        nn.Linear(128, 6),
        nn.Sigmoid(),).to(device)
    optimizer = optim.Adam(encoder.parameters(), lr=LEARNING_RATE)
    return encoder,decoder,optimizer

In [20]:
dataset.self_dataloader_init(batch_size=BATCH_SIZE)

In [62]:
# 计算 batch data 两两距离,利用 mse
# 使用 transfer function
def distance_matrix_4(data,mean,eps=1e-8,k=3):
    x1 = data.unsqueeze(1).repeat(1, data.shape[0], 1)
    x2 = data.unsqueeze(0)
    dist = (x1 - x2).pow(2).sum(2)
    dist = (dist+eps).sqrt()
    r = data.shape[0]
    mask = torch.triu(torch.ones([r,r]),diagonal=1) 
    # 传统方式
    # dist[mask==1] = (dist[mask==1])/g_mean
    ## 1. test 距离小于1的,就套用(1/(X+1)**6)+1,让它的增幅变大
#     dist[mask==1] = (dist[mask==1])/1.147
#     dist[dist<=1] = (1/-(dist[dist<=1]+1)**6)+1
    # 2. test transfer function, function 为 1/(-(X+0.9045)**3)+1.5,
    dist[mask==1] = 1/(-(dist[mask==1]+0.9045)**k)+1.5
#     dist[mask==1] = dist[mask==1]/mean
    # 3. test 线性转换
    # dist = dist*1.2
    # dist[dist>1.5] = 1.5
    # dist[mask==1] = dist[mask==1]/0.9628
    return dist[mask==1]

In [63]:
# 1/(-(distance_matrix+0.9045)**test_k)+1.5
test_k = 4
distance_matrix = torch.load('distance_map/bike_sharing_pre_compute_distance_matrix_temp.pth')
distance_matrix = 1/(-(distance_matrix+0.9045)**test_k)+1.5
dist_mean = distance_matrix.mean()    
encoder,decoder,optimizer = model()
for epoch in tqdm_notebook(range(242)):
    for i in range(times+1):
        batch_data = dataset.batch_data(BATCH_SIZE)
        batch_data = batch_data.to(device)
        optimizer.zero_grad()
        x = batch_data[:, :6]
        y = batch_data[:, 6:]
        z = encoder(x)
        recons = decoder(z)
        xd = distance_matrix_4(data=x,mean=dist_mean,k=test_k)
        zd = distance_matrix_Z(data=z)
        loss = torch.abs(xd-zd).mean()
        loss.backward()
        optimizer.step()
    print(loss.item())
torch.save(encoder.state_dict(),"./transfer_weight/"+str(test_k)+"_2.pth")

0.5249593257904053
0.49594366550445557
0.3507770001888275
0.37143105268478394
0.35478919744491577
0.3498307168483734
0.3588559627532959
0.3331696391105652
0.3669494390487671
0.36101508140563965
0.34971198439598083
0.3296913504600525
0.33409756422042847
0.36219683289527893
0.3463627099990845
0.3220687210559845
0.3335493206977844
0.3757789433002472
0.32909947633743286
0.342632919549942
0.3296358287334442
0.34972140192985535
0.33277222514152527
0.33739081025123596
0.3406006991863251
0.3262985646724701
0.3518047332763672
0.34299299120903015
0.3131205439567566
0.3394756019115448
0.30963724851608276
0.3453725278377533
0.33675462007522583
0.34041622281074524
0.3320659399032593
0.3289574682712555
0.3324785530567169
0.3252011239528656
0.3220967650413513
0.32683831453323364
0.3381052613258362
0.3429971933364868
0.3164338171482086
0.3336624503135681
0.311598539352417
0.34103986620903015
0.3184354305267334
0.31335946917533875
0.3353843688964844
0.32124063372612
0.32715073227882385
0.33267939090728

In [38]:
# 1/(-(distance_matrix+0.9225)**test_k)+1.5
test_k = 5
distance_matrix = torch.load('distance_map/bike_sharing_pre_compute_distance_matrix_temp.pth')
distance_matrix = 1/(-(distance_matrix+0.9225)**test_k)+1.5
dist_mean = distance_matrix.mean()    
encoder,decoder,optimizer = model()
for epoch in tqdm_notebook(range(242)):
    for i in range(times+1):
        batch_data = dataset.batch_data(BATCH_SIZE)
        batch_data = batch_data.to(device)
        optimizer.zero_grad()
        x = batch_data[:, :6]
        y = batch_data[:, 6:]
        z = encoder(x)
        recons = decoder(z)
        xd = distance_matrix_Ｘ(data=x,mean=dist_mean,k=test_k)
        zd = distance_matrix_Z(data=z)
        loss = torch.abs(xd-zd).mean()
        loss.backward()
        optimizer.step()
    print(loss.item())
torch.save(encoder.state_dict(),"./transfer_weight/"+str(test_k)+".pth")

0.39419251680374146
0.3648901879787445
0.2536694407463074
0.2539522051811218
0.23480595648288727
0.23129241168498993
0.24039430916309357
0.22992344200611115
0.24936984479427338
0.24767252802848816
0.22739678621292114
0.23357932269573212
0.2270403653383255
0.24924591183662415
0.23521560430526733
0.2371300905942917
0.24862752854824066
0.23920296132564545
0.22714638710021973
0.24004937708377838
0.23759348690509796
0.23299412429332733
0.22687794268131256
0.23478229343891144
0.2537650465965271
0.22898335754871368
0.23417706787586212
0.24196380376815796
0.25309082865715027
0.23398467898368835
0.22570115327835083
0.23072050511837006
0.22889170050621033
0.24854813516139984
0.23649241030216217
0.2331717163324356
0.23586682975292206
0.23080231249332428
0.23317059874534607
0.22863753139972687
0.22459359467029572
0.21927429735660553
0.2392423152923584
0.24023237824440002
0.23047170042991638
0.21423950791358948
0.22442957758903503
0.22515513002872467
0.2299155741930008
0.22847577929496765
0.2344245

In [39]:
# 1/(-(distance_matrix+0.935)**test_k)+1.5
test_k = 6
distance_matrix = torch.load('distance_map/bike_sharing_pre_compute_distance_matrix_temp.pth')
distance_matrix = 1/(-(distance_matrix+0.935)**test_k)+1.5
dist_mean = distance_matrix.mean()    
encoder,decoder,optimizer = model()
for epoch in tqdm_notebook(range(242)):
    for i in range(times+1):
        batch_data = dataset.batch_data(BATCH_SIZE)
        batch_data = batch_data.to(device)
        optimizer.zero_grad()
        x = batch_data[:, :6]
        y = batch_data[:, 6:]
        z = encoder(x)
        recons = decoder(z)
        xd = distance_matrix_Ｘ(data=x,mean=dist_mean,k=test_k)
        zd = distance_matrix_Z(data=z)
        loss = torch.abs(xd-zd).mean()
        loss.backward()
        optimizer.step()
    print(loss.item())
torch.save(encoder.state_dict(),"./transfer_weight/"+str(test_k)+".pth")

0.38701552152633667
0.25972259044647217
0.25358083844184875
0.253558486700058
0.25347644090652466
0.251624196767807
0.2434588223695755
0.25002357363700867
0.24725064635276794
0.25877290964126587
0.2531931698322296
0.24862155318260193
0.2611217200756073
0.24888715147972107
0.24928990006446838
0.24186883866786957
0.24041256308555603
0.24961870908737183
0.2708853483200073
0.24880897998809814
0.2415132075548172
0.25588488578796387
0.24289646744728088
0.25134575366973877
0.25597548484802246
0.24529649317264557
0.24060270190238953
0.2538103461265564
0.2513377070426941
0.254606693983078
0.2503969669342041
0.23359829187393188
0.2578122615814209
0.24000416696071625
0.23358775675296783
0.2329888790845871
0.22876477241516113
0.2426350712776184
0.23704662919044495
0.25005823373794556
0.25890156626701355
0.2569742500782013
0.25447162985801697
0.25401487946510315
0.23683153092861176
0.24992108345031738
0.2556568384170532
0.25416871905326843
0.23019206523895264
0.2444610744714737
0.23781628906726837


In [40]:
# 1/(-(distance_matrix+0.945)**test_k)+1.5
test_k = 7
distance_matrix = torch.load('distance_map/bike_sharing_pre_compute_distance_matrix_temp.pth')
distance_matrix = 1/(-(distance_matrix+0.945)**test_k)+1.5
dist_mean = distance_matrix.mean()    
encoder,decoder,optimizer = model()
for epoch in tqdm_notebook(range(242)):
    for i in range(times+1):
        batch_data = dataset.batch_data(BATCH_SIZE)
        batch_data = batch_data.to(device)
        optimizer.zero_grad()
        x = batch_data[:, :6]
        y = batch_data[:, 6:]
        z = encoder(x)
        recons = decoder(z)
        xd = distance_matrix_Ｘ(data=x,mean=dist_mean,k=test_k)
        zd = distance_matrix_Z(data=z)
        loss = torch.abs(xd-zd).mean()
        loss.backward()
        optimizer.step()
    print(loss.item())
torch.save(encoder.state_dict(),"./transfer_weight/"+str(test_k)+".pth")

0.40075623989105225
0.40603819489479065
0.271842896938324
0.26250678300857544
0.26453903317451477
0.2584785223007202
0.25423774123191833
0.26016131043434143
0.2902369797229767
0.2660568058490753
0.2527132034301758
0.25992143154144287
0.24726241827011108
0.2701962888240814
0.26116520166397095
0.2561728060245514
0.24450987577438354
0.25831085443496704
0.26327750086784363
0.25759944319725037
0.2556294798851013
0.24611227214336395
0.24379460513591766
0.2502419650554657
0.2636450529098511
0.2545088529586792
0.25857752561569214
0.25147464871406555
0.25669801235198975
0.25186142325401306
0.2618468999862671
0.2509872317314148
0.24703693389892578
0.259032666683197
0.2449391633272171
0.2581392526626587
0.2385614663362503
0.2536073625087738
0.2618141770362854
0.2420005202293396
0.2582918703556061
0.2515343427658081
0.24947857856750488
0.2496691644191742
0.2607229948043823
0.23880554735660553
0.2509017884731293
0.24202503263950348
0.27976173162460327
0.24955159425735474
0.22419892251491547
0.24698

In [41]:
# 1/(-(distance_matrix+0.955)**test_k)+1.5
test_k = 8
distance_matrix = torch.load('distance_map/bike_sharing_pre_compute_distance_matrix_temp.pth')
distance_matrix = 1/(-(distance_matrix+0.955)**test_k)+1.5
dist_mean = distance_matrix.mean()    
encoder,decoder,optimizer = model()
for epoch in tqdm_notebook(range(242)):
    for i in range(times+1):
        batch_data = dataset.batch_data(BATCH_SIZE)
        batch_data = batch_data.to(device)
        optimizer.zero_grad()
        x = batch_data[:, :6]
        y = batch_data[:, 6:]
        z = encoder(x)
        recons = decoder(z)
        xd = distance_matrix_Ｘ(data=x,mean=dist_mean,k=test_k)
        zd = distance_matrix_Z(data=z)
        loss = torch.abs(xd-zd).mean()
        loss.backward()
        optimizer.step()
    print(loss.item())
torch.save(encoder.state_dict(),"./transfer_weight/"+str(test_k)+".pth")

0.42157360911369324
0.327065646648407
0.28338637948036194
0.28856709599494934
0.2563064396381378
0.2707861363887787
0.279805064201355
0.2823883593082428
0.26453763246536255
0.28223076462745667
0.27612125873565674
0.26303502917289734
0.2590923011302948
0.26864805817604065
0.27103787660598755
0.2769307494163513
0.2604672908782959
0.26854804158210754
0.2590242624282837
0.26640233397483826
0.26667267084121704
0.2694694399833679
0.265197217464447
0.2735707759857178
0.2635638117790222
0.2727683186531067
0.27200961112976074
0.26223024725914
0.2734230160713196
0.2702113687992096
0.2724582254886627
0.2640748918056488
0.2570023536682129
0.26373031735420227
0.2770557403564453
0.30653584003448486
0.2607865333557129
0.25831785798072815
0.26005828380584717
0.2633240222930908
0.25407567620277405
0.27091971039772034
0.2549895644187927
0.2573767304420471
0.26305481791496277
0.2534749507904053
0.2562970817089081
0.26112914085388184
0.27368587255477905
0.26185083389282227
0.26106131076812744
0.2521266043

In [64]:
# 计算 batch data 两两距离,利用 mse
# 使用 transfer function
def distance_matrix_9(data,mean,eps=1e-8,k=3):
    x1 = data.unsqueeze(1).repeat(1, data.shape[0], 1)
    x2 = data.unsqueeze(0)
    dist = (x1 - x2).pow(2).sum(2)
    dist = (dist+eps).sqrt()
    r = data.shape[0]
    mask = torch.triu(torch.ones([r,r]),diagonal=1) 
    # 2. test transfer function, function 为 1/(-(X+0.956)**3)+1.5,
    dist[mask==1] = 1/(-(dist[mask==1]+0.956)**k)+1.5
#     dist[mask==1] = dist[mask==1]/mean
    return dist[mask==1]

In [65]:
test_k = 9
distance_matrix = torch.load('distance_map/bike_sharing_pre_compute_distance_matrix_temp.pth')
distance_matrix = 1/(-(distance_matrix+0.956)**test_k)+1.5
dist_mean = distance_matrix.mean()    
encoder,decoder,optimizer = model()
for epoch in tqdm_notebook(range(242)):
    for i in range(times+1):
        batch_data = dataset.batch_data(BATCH_SIZE)
        batch_data = batch_data.to(device)
        optimizer.zero_grad()
        x = batch_data[:, :6]
        y = batch_data[:, 6:]
        z = encoder(x)
        recons = decoder(z)
        xd = distance_matrix_9(data=x,mean=dist_mean,k=test_k)
        zd = distance_matrix_Z(data=z)
        loss = torch.abs(xd-zd).mean()
        loss.backward()
        optimizer.step()
    print(loss.item())
torch.save(encoder.state_dict(),"./transfer_weight/"+str(test_k)+"_2.pth")

0.6765126585960388
0.705557107925415
0.6757978796958923
0.6882428526878357
0.48640042543411255
0.47060123085975647
0.46797195076942444
0.4440307915210724
0.44753187894821167
0.46825557947158813
0.4486783742904663
0.46577462553977966
0.466071754693985
0.4631192088127136
0.460102379322052
0.45654475688934326
0.44948670268058777
0.4565839171409607
0.44388988614082336
0.45642194151878357
0.46456378698349
0.4441669285297394
0.44804641604423523
0.4407932162284851
0.4686264097690582
0.43775227665901184
0.4547906219959259
0.4412879943847656
0.4619204103946686
0.452261358499527
0.43613576889038086
0.44849494099617004
0.45452186465263367
0.42531096935272217
0.43233776092529297
0.43308791518211365
0.4529666304588318
0.435981422662735
0.4415132999420166
0.4394669830799103
0.4492415487766266
0.43794023990631104
0.4313119649887085
0.43365082144737244
0.4459610879421234
0.4458470344543457
0.42885491251945496
0.4468531906604767
0.44637754559516907
0.43127745389938354
0.42126187682151794
0.460113674402

In [67]:
# 计算 batch data 两两距离,利用 mse
# 使用 transfer function
def distance_matrix_linear(data,mean,rate,eps=1e-8):
    x1 = data.unsqueeze(1).repeat(1, data.shape[0], 1)
    x2 = data.unsqueeze(0)
    dist = (x1 - x2).pow(2).sum(2)
    dist = (dist+eps).sqrt()
    r = data.shape[0]
    mask = torch.triu(torch.ones([r,r]),diagonal=1) 
    # linear transfer [dist > 1.5]*(rate) , [dist <= 1.5]*[2-rate]
    high = torch.ones(r,r).to(device)
    low = torch.ones(r,r).to(device)
    high[dist>1.5] = rate
    low[dist<=1.5] = 2-rate
    dist = high * dist
    dist = low * dist
#     dist[mask==1] = dist[mask==1]/mean
    return dist[mask==1]

In [69]:
# linear transfer function 
distance_matrix = torch.load('distance_map/bike_sharing_pre_compute_distance_matrix_temp.pth')
r = distance_matrix.shape[0]
rate = distance_matrix[distance_matrix <= 1.5].shape[0]/r
high = torch.ones(r)
low = torch.ones(r)
high[distance_matrix>1.5] = rate
low[distance_matrix<=1.5] = (2-rate)
distance_matrix = low * distance_matrix
distance_matrix = high * distance_matrix
distance_mean = distance_matrix.mean() 
encoder,decoder,optimizer = model()
print(rate, 2-rate)
for epoch in tqdm_notebook(range(242)):
    for i in range(times+1):
        batch_data = dataset.batch_data(BATCH_SIZE)
        batch_data = batch_data.to(device)
        optimizer.zero_grad()
        x = batch_data[:, :6]
        y = batch_data[:, 6:]
        z = encoder(x)
        recons = decoder(z)
        xd = distance_matrix_linear(data=x,mean=dist_mean,rate=rate)
        zd = distance_matrix_Z(data=z)
        loss = torch.abs(xd-zd).mean()
        loss.backward()
        optimizer.step()
    print(loss.item())
torch.save(encoder.state_dict(),"./transfer_weight/linear_2.pth")

0.8755008165860497 1.1244991834139504


0.21295137703418732
0.1035173237323761
0.11179976165294647
0.08295971155166626
0.09644968807697296
0.10356561094522476
0.10965026915073395
0.08813593536615372
0.08827624469995499
0.0914609357714653
0.09983044117689133
0.10530823469161987
0.09930002689361572
0.09116999059915543
0.1012340635061264
0.09814081341028214
0.09335639327764511
0.07490188628435135
0.09834667295217514
0.09161520004272461
0.07724089175462723
0.0868489146232605
0.0810919925570488
0.10305099189281464
0.08818396180868149
0.08564697206020355
0.08499807864427567
0.07795438915491104
0.08851344138383865
0.08913463354110718
0.08375231921672821
0.09043533354997635
0.09048120677471161
0.0864025354385376
0.0893237367272377
0.08304034173488617
0.07849881052970886
0.0875185877084732
0.08187946677207947
0.09527414292097092
0.07880911976099014
0.09349474310874939
0.09279521554708481
0.07938794046640396
0.08456819504499435
0.0975353941321373
0.09611834585666656
0.08317623287439346
0.08134587109088898
0.07415492832660675
0.0991919

In [ ]:
for test_k in tqdm_notebook(range(10)):
    distance_matrix = torch.load('distance_map/bike_sharing_pre_compute_distance_matrix_temp.pth')
    distance_matrix = 1/(-(distance_matrix+0.874)**test_k)+1.5
    dist_mean = distance_matrix.mean()    
    encoder,decoder,optimizer = model()
    for epoch in tqdm_notebook(range(2000)):
        for i in range(times+1):
            batch_data = dataset.batch_data(BATCH_SIZE)
            batch_data = batch_data.to(device)
            optimizer.zero_grad()
            x = batch_data[:, :6]
            y = batch_data[:, 6:]
            z = encoder(x)
            recons = decoder(z)
            xd = distance_matrix_Ｘ(data=x,mean=dist_mean,k=test_k)
            zd = distance_matrix_Z(data=z)
            loss = torch.abs(xd-zd).mean()
            loss.backward()
            optimizer.step()
        print(loss.item())
    torch.save(encoder.state_dict(),"./transfer_weight/"+str(test_k)+".pth")

0.47483736276626587
0.3593788743019104
0.3491426706314087
0.34635841846466064
0.34482553601264954
0.346670925617218
0.3421398103237152
0.3419560492038727
0.3381214439868927
0.34949639439582825
0.3419797122478485
0.3441842198371887
0.3377823531627655
0.33852022886276245
0.354480504989624
